<a href="https://colab.research.google.com/github/dlwogns/ML-DL/blob/main/GoDaddy%20-%20Microbusiness%20Density%20Forecasting/ARIMA_godaddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARIMA 
## reference
https://www.kaggle.com/code/tanmay111999/gdmbf-ar-ma-arma-arima-sarima-auto-arima <br>
https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm import tqdm

from itertools import combinations
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima.model import ARIMA as ARIMA
import statsmodels.api as sm
import statsmodels.tsa.api as smt

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/train.csv")
test = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/test.csv")
census = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/census_starter.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/data/GoDaddy/sample_submission.csv")

In [ ]:
data = train[['first_day_of_month', 'microbusiness_density']].copy(deep = True)
data['Date'] = pd.to_datetime(data['first_day_of_month'])
data = data.drop(columns = 'first_day_of_month')
data = data.set_index('Date')
data

,microbusiness_density
Date,
2019-08-01,3.007682
2019-09-01,2.884870
2019-10-01,3.055843
2019-11-01,2.993233
2019-12-01,2.993233
...,...
2022-06-01,1.803249
2022-07-01,1.803249
2022-08-01,1.785395


In [ ]:
# cfip과 censor data까지 고려해주어야 하지만, 여기서 추세를 파악하기 위해 일단은 
# 전체의 평균을 내버렸다.
data = data.groupby(data.index).mean()
data

,microbusiness_density
Date,
2019-08-01,3.749621
2019-09-01,3.696846
2019-10-01,3.778263
2019-11-01,3.722510
2019-12-01,3.717284
2020-01-01,3.689041
2020-02-01,3.635546
2020-03-01,3.648742
2020-04-01,3.700048


In [ ]:
model = ARIMA(data['microbusiness_density'], order = (0,0,0))
model_fit = model.fit()
print(model_fit.summary())

                                 SARIMAX Results                                 
Dep. Variable:     microbusiness_density   No. Observations:                   39
Model:                             ARIMA   Log Likelihood                  34.482
Date:                   Sun, 12 Feb 2023   AIC                            -64.964
Time:                           04:48:51   BIC                            -61.637
Sample:                       08-01-2019   HQIC                           -63.770
                            - 10-01-2022                                         
Covariance Type:                     opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8177      0.016    236.742      0.000       3.786       3.849
sigma2         0.0100      0.004      2.835      0.005       0.003       0.017
Ljung-Box (L1) (Q):         

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [ ]:
test['first_day_of_month'] = pd.to_datetime(test['first_day_of_month'])
df1 = test.copy(deep = True)
df1 = df1.set_index('first_day_of_month')
df1 = df1.groupby(df1.index).mean()
df1

,cfips
first_day_of_month,
2022-11-01,30376.03764
2022-12-01,30376.03764
2023-01-01,30376.03764
2023-02-01,30376.03764
2023-03-01,30376.03764
2023-04-01,30376.03764
2023-05-01,30376.03764
2023-06-01,30376.03764


In [ ]:
history_f = [x for x in data['microbusiness_density']]
f1 = []
for t in tqdm(range(len(df1))):
  model = ARIMA(history_f, order = (0,1,0))
  model_fit = model.fit()
  output = model_fit.forecast()[0]
  history_f.append(output)
  f1.append(output)

df1['forecast'] = 0
for i in range(len(f1)):
  df1.iloc[i, 1] = f1[i]

df1
  

  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 12%|█▎        | 1/8 [00:00<00:00,  9.03it/s]/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 38%|███▊      | 3/8 [00:00<00:00, 13.44it/s]/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
100%|██████████| 8/8 [00:00<00:00, 19.99it/s]


,cfips,forecast
first_day_of_month,,
2022-11-01,30376.03764,4.006963
2022-12-01,30376.03764,4.006963
2023-01-01,30376.03764,4.006963
2023-02-01,30376.03764,4.006963
2023-03-01,30376.03764,4.006963
2023-04-01,30376.03764,4.006963
2023-05-01,30376.03764,4.006963
2023-06-01,30376.03764,4.006963


In [ ]:
history_p = [x for x in data['microbusiness_density']]
p1 = []
for t in tqdm(range(len(df1))):
  model = ARIMA(history_p, order=(0,1,0))
  model_fit = model.fit()
  output = model_fit.predict(start = len(history_p), end = len(history_p))[0]
  history_p.append(output)
  p1.append(output)

df1['forecast'] = 0
for i in range(len(p1)):
  df1.iloc[i, 1] = p1[i]

df1

  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 25%|██▌       | 2/8 [00:00<00:00, 16.25it/s]/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


[3.749621441269219, 3.6968457058593303, 3.778263448494099, 3.7225099674255184, 3.717283839049123, 3.6890410975317387, 3.635545970995853, 3.648742413388198, 3.700047868635407, 3.716288394201595, 3.723050887348006, 3.742933533182137, 3.764327914706858, 3.7715576913001594, 3.7695249484357256, 3.7589744711598088, 3.7446496386877195, 3.7114203925763958, 3.785175065077193, 3.8134347899416263, 3.8355827308602874, 3.8345704536066987, 3.8351558110899524, 3.8450638751218498, 3.8603166405827753, 3.8611703491952154, 3.8768024541744817, 3.9105802798937797, 3.924641756238277, 3.9021765981987238, 3.918186123199362, 3.955752844950558, 3.9594407598596493, 3.9417988590558215, 3.902961102137161, 3.965593065027113, 3.953797914889952, 3.959357988985646, 4.006963110398724, 4.006963110398724]
[3.749621441269219, 3.6968457058593303, 3.778263448494099, 3.7225099674255184, 3.717283839049123, 3.6890410975317387, 3.635545970995853, 3.648742413388198, 3.700047868635407, 3.716288394201595, 3.723050887348006, 3.7429

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 75%|███████▌  | 6/8 [00:00<00:00, 13.66it/s]

[3.749621441269219, 3.6968457058593303, 3.778263448494099, 3.7225099674255184, 3.717283839049123, 3.6890410975317387, 3.635545970995853, 3.648742413388198, 3.700047868635407, 3.716288394201595, 3.723050887348006, 3.742933533182137, 3.764327914706858, 3.7715576913001594, 3.7695249484357256, 3.7589744711598088, 3.7446496386877195, 3.7114203925763958, 3.785175065077193, 3.8134347899416263, 3.8355827308602874, 3.8345704536066987, 3.8351558110899524, 3.8450638751218498, 3.8603166405827753, 3.8611703491952154, 3.8768024541744817, 3.9105802798937797, 3.924641756238277, 3.9021765981987238, 3.918186123199362, 3.955752844950558, 3.9594407598596493, 3.9417988590558215, 3.902961102137161, 3.965593065027113, 3.953797914889952, 3.959357988985646, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724]
[3.749621441269219, 3.6968457058593303, 3.778263448494099, 3.7225099674255184, 3.717283839049123, 3.6890410975317387, 3.635545970995853, 3.648742413388198, 3.7000

100%|██████████| 8/8 [00:00<00:00, 14.39it/s]

[3.749621441269219, 3.6968457058593303, 3.778263448494099, 3.7225099674255184, 3.717283839049123, 3.6890410975317387, 3.635545970995853, 3.648742413388198, 3.700047868635407, 3.716288394201595, 3.723050887348006, 3.742933533182137, 3.764327914706858, 3.7715576913001594, 3.7695249484357256, 3.7589744711598088, 3.7446496386877195, 3.7114203925763958, 3.785175065077193, 3.8134347899416263, 3.8355827308602874, 3.8345704536066987, 3.8351558110899524, 3.8450638751218498, 3.8603166405827753, 3.8611703491952154, 3.8768024541744817, 3.9105802798937797, 3.924641756238277, 3.9021765981987238, 3.918186123199362, 3.955752844950558, 3.9594407598596493, 3.9417988590558215, 3.902961102137161, 3.965593065027113, 3.953797914889952, 3.959357988985646, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724, 4.006963110398724]


,cfips,forecast
first_day_of_month,,
2022-11-01,30376.03764,4.006963
2022-12-01,30376.03764,4.006963
2023-01-01,30376.03764,4.006963
2023-02-01,30376.03764,4.006963
2023-03-01,30376.03764,4.006963
2023-04-01,30376.03764,4.006963
2023-05-01,30376.03764,4.006963
2023-06-01,30376.03764,4.006963
